In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import astropy
from astropy.io import fits

import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import hdbscan

In [2]:
hdul = fits.open(
    'http://gal-03.sai.msu.ru/~vtoptun/photometry/rcsed_v2_clean.fits',
    memmap=astropy.io.fits.Conf.use_memmap.defaultvalue,
    lazy_load_hdus=True,
)

y = pd.read_csv('rcsed_iGrID.csv')

In [3]:
sdss_indx = list(y[~y.iGrID.isna()].index)
y = y.loc[sdss_indx,:].to_numpy().flatten()
sdss_labels = pd.Series(y)

cols, data = hdul[1].columns, hdul[1].data[sdss_indx]

In [4]:
hdul.close()
del hdul

In [5]:
DATA = pd.DataFrame(np.array(data).byteswap().newbyteorder()) 

In [6]:
del data

In [7]:
DATA = DATA.select_dtypes(include=['float32', 'float64']).apply(pd.to_numeric,downcast='float')
x = np.array(DATA[['ra','dec','z']])
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [8]:
hdbScan = hdbscan.hdbscan_.HDBSCAN().fit(x)

rcsed_labels = hdbScan.labels_
for i in range(len(rcsed_labels)):
    if rcsed_labels[i]==-1:
        rcsed_labels[i]=i+5000000

true = y
pred = rcsed_labels

fms = round(sklearn.metrics.fowlkes_mallows_score(true, pred),5)
ars = round(sklearn.metrics.adjusted_rand_score(true, pred),5)
nmi = round(sklearn.metrics.normalized_mutual_info_score(true, pred),5)

print('FMS = ', fms)
print('ARS = ', ars)
print('NMI = ', nmi)

FMS =  0.33041
ARS =  0.3134
NMI =  0.93437
